In [ ]:
CURRENT_STATE = (0, 0, 0) # x, y, theta
GOAL_POSITION = (0, 0) # x, y
CELL_SIZE = 0.1
SDF_ORIGIN = (0, 0)

In [1]:
import theseus as th
from planner import MotionPlannerObjective
import torch

dtype = torch.double
device = torch.device("cuda:0")
objective = MotionPlannerObjective(
    total_time=60.0,
    horizon=30,
    current_velocity=torch.tensor([0.0, 0.0], dtype=dtype),
    x_velocity_bounds=torch.tensor([-5, 5], dtype=dtype),
    y_velocity_bounds=torch.tensor([-5, 5], dtype=dtype),
    x_acceleration_bounds=torch.tensor([-5, 5], dtype=dtype),
    y_acceleration_bounds=torch.tensor([-5, 5], dtype=dtype),
    robot_radius=0.5, # REPLACE WITH ACTUAL RADIUS
    safety_distance=0.1, # REPLACE WITH ACTUAL SAFETY DISTANCE
    local_map_size=128, # REPLACE WITH ACTUAL MAP SIZE
)

optimizer = th.LevenbergMarquardt(
    objective,
    th.CholeskyDenseSolver,
    max_iterations=50,
    step_size=1.0,
)
motion_planner = th.TheseusLayer(optimizer)
motion_planner.to(torch.device("cuda:0"), dtype=torch.double)

In [2]:
def get_straight_line_inputs(start: torch.Tensor, goal: torch.Tensor, total_time, horizon):
    # Returns a dictionary with state and acceleration variable names associated to a 
    # straight line trajectory between start and goal in SE(2) space
    
    # Calculate start and goal positions
    start_pos = start[..., :2]  # Extracting the 2D position from SE(2)
    goal_pos = goal  # Assuming goal is already a 2D point
    
    # Calculate distances and average accelerations for each batch element
    start_goal_dist = goal_pos - start_pos  # Shape: (batch_size, 2)
    avg_acc = 2 * start_goal_dist / (total_time**2)  # Shape: (batch_size, 2)
    unit_horizon = start_goal_dist / (horizon - 1)  # Shape: (batch_size, 2)
    
    # Create input dictionary
    input_dict = {}
    
    for i in range(horizon + 1):
        state_i = start_pos + unit_horizon * i  # Shape: (batch_size, 2)
        input_dict[f"state_{i}"] = torch.concat([state_i, start[..., 2:]], dim=-1)  # Concatenate the 2D position with the orientation
        
        if i == horizon:
            continue
        
        if i == 0 or i == horizon - 1:
            acceleration_i = torch.zeros_like(avg_acc)  # Zero acceleration at start and end
        else:
            acceleration_i = avg_acc
        
        input_dict[f"acceleration_{i}"] = acceleration_i
        
    return input_dict

In [5]:
planner_inputs = {
    'sdf_data': torch.zeros([128, 128], dtype=dtype).unsqueeze(0).to(device), # REPLACE WITH SDF DATA
    'sdf_origin': torch.tensor([SDF_ORIGIN[0], SDF_ORIGIN[1]], dtype=dtype).unsqueeze(0).to(device),
    'cell_size': torch.tensor([CELL_SIZE], dtype=dtype).unsqueeze(0).to(device),
    'current_state': torch.tensor([CURRENT_STATE[0], CURRENT_STATE[1], torch.cos(CURRENT_STATE[2]), torch.sin(CURRENT_STATE[2])], dtype=dtype).unsqueeze(0).to(device),
    'goal_position': torch.tensor([GOAL_POSITION[0], GOAL_POSITION[1]], dtype=dtype).unsqueeze(0).to(device),
}
planner_inputs.update(get_straight_line_inputs(
    planner_inputs['current_state'], planner_inputs['goal_position'], 60.0, 30))

In [6]:
with torch.no_grad():
    final_values, info = motion_planner.forward(
        planner_inputs,
        optimizer_kwargs={
            "track_best_solution": True,
            "verbose": True,
            "damping": 0.1,
        }
    )

Nonlinear optimizer. Iteration: 0. Error: 12628911.602194458
Nonlinear optimizer. Iteration: 1. Error: 3291685.0078109805
Nonlinear optimizer. Iteration: 2. Error: 3291636.0657504573
Nonlinear optimizer. Iteration: 3. Error: 3291635.5264588045
Nonlinear optimizer. Iteration: 4. Error: 3291635.527780474


In [7]:
info.best_solution

{'state_1': tensor([[4.5589, 4.5566, 1.0000, 0.0000]], dtype=torch.float64),
 'state_2': tensor([[7.5862, 7.6085, 1.0000, 0.0000]], dtype=torch.float64),
 'state_3': tensor([[9.3742, 9.4093, 1.0000, 0.0000]], dtype=torch.float64),
 'state_4': tensor([[10.2338, 10.2674,  1.0000,  0.0000]], dtype=torch.float64),
 'state_5': tensor([[10.5139, 10.5387,  1.0000,  0.0000]], dtype=torch.float64),
 'state_6': tensor([[10.4954, 10.5105,  1.0000,  0.0000]], dtype=torch.float64),
 'state_7': tensor([[10.3635, 10.3710,  1.0000,  0.0000]], dtype=torch.float64),
 'state_8': tensor([[10.2201, 10.2227,  1.0000,  0.0000]], dtype=torch.float64),
 'state_9': tensor([[10.1081, 10.1082,  1.0000,  0.0000]], dtype=torch.float64),
 'state_10': tensor([[10.0367, 10.0358,  1.0000,  0.0000]], dtype=torch.float64),
 'state_11': tensor([[9.9993, 9.9982, 1.0000, 0.0000]], dtype=torch.float64),
 'state_12': tensor([[9.9847, 9.9838, 1.0000, 0.0000]], dtype=torch.float64),
 'state_13': tensor([[9.9828, 9.9822, 1.0000,